# Multi-Layer Feed Forward Neural Network To Predict Diabetes
## Using PyTorch

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
# Load the dataset using Pandas
data = pd.read_csv('diabetes.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [3]:
# For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). 
# For y: Extract out the last column (which is the label)
# Convert both to numpy using the .values method
x = data.iloc[:,0:-1].values
y_string= list(data.iloc[:,-1])

In [4]:
# Lets have a look some samples from our data
print(x[:3])
print(y_string[:3])

[[  6.  148.   72.   35.    0.   33.6  50. ]
 [  1.   85.   66.   29.    0.   26.6  31. ]
 [  8.  183.   64.    0.    0.   23.3  32. ]]
['positive', 'negative', 'positive']


In [5]:
# Our neural network only understand numbers! So convert the string to labels
y_int = []
for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [6]:
# Now convert to an array
y = np.array(y_int, dtype = 'float64')

### $x^{\prime}=\frac{x-\mu}{\sigma}$

In [7]:
# Feature Normalization. All features should have the same range of values (-1,1)
sc = StandardScaler()
x = sc.fit_transform(x)

In [8]:
# Now we convert the arrays to PyTorch tensors
x = torch.tensor(x)
# We add an extra dimension to convert this array to 2D
y = torch.tensor(y).unsqueeze(1)

In [9]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


### Let's look at the first few elements of the $x$ and $y$ tensors

In [10]:
x[:10,:]

tensor([[ 0.6399,  0.8483,  0.1496,  0.9073, -0.6929,  0.2040,  1.4260],
        [-0.8449, -1.1234, -0.1605,  0.5309, -0.6929, -0.6844, -0.1907],
        [ 1.2339,  1.9437, -0.2639, -1.2882, -0.6929, -1.1033, -0.1056],
        [-0.8449, -0.9982, -0.1605,  0.1545,  0.1233, -0.4940, -1.0415],
        [-1.1419,  0.5041, -1.5047,  0.9073,  0.7658,  1.4097, -0.0205],
        [ 0.3430, -0.1532,  0.2530, -1.2882, -0.6929, -0.8113, -0.2758],
        [-0.2510, -1.3425, -0.9877,  0.7191,  0.0712, -0.1260, -0.6161],
        [ 1.8278, -0.1845, -3.5726, -1.2882, -0.6929,  0.4198, -0.3608],
        [-0.5479,  2.3819,  0.0462,  1.5346,  4.0219, -0.1894,  1.6813],
        [ 1.2339,  0.1285,  1.3904, -1.2882, -0.6929, -4.0605,  1.7663]],
       dtype=torch.float64)

In [11]:
y[:10,:]

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.]], dtype=torch.float64)

In [12]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

#### Create the `dataset` object...

In [13]:
dataset = Dataset(x,y)

In [14]:
len(dataset)

768

In [15]:
# Load the data to your dataloader for batch processing and shuffling
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=32,
                                           shuffle=True)

In [16]:
# Let's have a look at the data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


![demo](https://user-images.githubusercontent.com/30661597/60379583-246e5e80-9a68-11e9-8b7f-a4294234c201.png)

In [17]:
# Now let's build the above network
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)   # fc = fully-connected
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)     # tanh activation on the internal layers
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)  # sigmoid activation on the output layer
        return out

$H_{p}(q)=-\frac{1}{N} \sum_{i=1}^{N} y_{i} \cdot \log \left(p\left(y_{i}\right)\right)+\left(1-y_{i}\right) \cdot \log \left(1-p\left(y_{i}\right)\right)$


cost = -(Y * torch.log(hypothesis) + (1 - Y) * torch.log(1 - hypothesis)).mean()

In [18]:
# Create the network (an object of the Net class)
neuralnet = Model(x.shape[1])
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True)   
# We will use SGD with momentum with a learning rate of 0.1
optimizer = torch.optim.SGD(neuralnet.parameters(), lr=0.1, momentum=0.9)

/opt/homebrew/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [19]:
# Train the network 
num_epochs = 200
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = neuralnet(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (we don't want to accumulate gradients)
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient
        optimizer.step()
        
    #Accuracy
    # Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()   # if output>0.5 then 1.0 else 0.0
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))  # epochs start at 0, so add 1

Epoch 1/200, Loss: 0.635, Accuracy: 0.562
Epoch 2/200, Loss: 0.497, Accuracy: 0.688
Epoch 3/200, Loss: 0.561, Accuracy: 0.719
Epoch 4/200, Loss: 0.397, Accuracy: 0.812
Epoch 5/200, Loss: 0.493, Accuracy: 0.781
Epoch 6/200, Loss: 0.577, Accuracy: 0.625
Epoch 7/200, Loss: 0.433, Accuracy: 0.781
Epoch 8/200, Loss: 0.452, Accuracy: 0.781
Epoch 9/200, Loss: 0.346, Accuracy: 0.812
Epoch 10/200, Loss: 0.405, Accuracy: 0.812
Epoch 11/200, Loss: 0.367, Accuracy: 0.812
Epoch 12/200, Loss: 0.352, Accuracy: 0.812
Epoch 13/200, Loss: 0.345, Accuracy: 0.844
Epoch 14/200, Loss: 0.542, Accuracy: 0.781
Epoch 15/200, Loss: 0.393, Accuracy: 0.781
Epoch 16/200, Loss: 0.387, Accuracy: 0.844
Epoch 17/200, Loss: 0.504, Accuracy: 0.750
Epoch 18/200, Loss: 0.393, Accuracy: 0.844
Epoch 19/200, Loss: 0.573, Accuracy: 0.688
Epoch 20/200, Loss: 0.529, Accuracy: 0.625
Epoch 21/200, Loss: 0.535, Accuracy: 0.688
Epoch 22/200, Loss: 0.391, Accuracy: 0.812
Epoch 23/200, Loss: 0.431, Accuracy: 0.781
Epoch 24/200, Loss: 